# Рекуррентные сети и моделирование последовательностей

## Последовательные данные

* Финансовые временные ряды.
* Аудиосигналы;
* Тексты;
* ...

## Идея рекуррентной сети

* Каждый новый элемент последовательности добавляет фрагмент информации к уже известной и обработанной моделью части данных.
* Модель имеет состояние, которое не перезаписывается заново, а обновляется при обработке нового члена последовательности.

Наример, чтение текста человеком: каждое новое слово не заставляет его забывать прочитанное ранее, а обновляет и дополняет "историю" и "содержимое памяти".

## Архитектура 

В нейронную сеть вводятся обратные связи.

<img src="img/rnn/rnn_intro.png" height="30%">

## Динамическая система

Пусть есть некоторая система, которая с течением времени меняет свое состояние:

$$s^{(t)} = f(s^{(t-1)}; \theta)$$

(Рекуррентное выражение)

Для конечного числа шагов $\tau$ это выражение можно развернуть (избавиться от рекурсии):

$$s^{(3)} = f(s^{(2)}; \theta) = f(f(s^{(1)}; \theta) ; \theta) $$

![Динамическая система $s^{(t)} = f(s^{(t-1)}; \theta)$ в виде развернутого графа вычислений. Каждая вершина -- состояние в момент $t$, функция $f$ отображает состояние в момент $t$ на состояние в момент $t+1$.](img/rnn/rnn_seq.png)
Динамическая система $s^{(t)} = f(s^{(t-1)}; \theta)$ в виде развернутого графа вычислений. 

* каждая вершина -- состояние в момент $t$, 
* функция $f$ отображает состояние в момент $t$ на состояние в момент $t+1$,
* одни и те же параметры ($f$ и $\theta$) используются на всех временных шагах.

### Нейронные сети с обратными связями - динамические системы

![Сеть с обратными связями (без внешних входных сигналов)](img/rnn/rnn_syst.png).

Сеть с обратными связями (без внешних входных сигналов)

### Системы, управляемые внешним сигналом

Путь $x(t)$ - внешний сигнал. Состояния системы, управляемой сигналом описываются рекуррентным выражением:

$$s^{(t)} = f(s^{(t-1)}, x^{(t)}; \theta)$$.

Состояние зависит от *всей* прошлой входной последовательности.

В нейронных сетях состояния обычно представляются скрытыми блоками сети: 
$$h^{(t)} = f(h^{(t-1)}, x^{(t)}; \theta)$$.


<img src="img/rnn/rnn_intro.png" height="30%">

## Развертка рекуррентной сети

Для вычислений и подбора весов сети "разворачивают" во времени, при этом зависимости выходов сети от входных сигналов и параметров становится легче проследить.

![Пример развертки](img/rnn/rnn_unfold.png)
Принципиальная схема и ее развертка во времени. (Гудфеллоу Я. и др. "Глубокое обучение").

Рекуррентная сеть без выходов: обрабатывает входной сигнал $x$ и текущее состояние $h$, результат передается дальше во времени. 



### Преимущества развертки

1. Независимо от длины входной последовательности размер входа в сеть фиксирован, а не список входов/состояний переменной длины. 
(Без развертки было бы: $h^{(t)} = g^{(t)} (x^{(t)}, x^{(t-1)}, x^{(t-2)}, \dots, x^{(1)})$).
2. Функция перехода $f$ одинакова на каждом шаге.

Т.е. можно обучить *одну* модель $f$, работающую на всех временных шагах, а не обучать отдельно для каждого шага.

## Распространенные схемы рекуррентных сетей

1. Сети, порождающие выход на каждом временном шаге, и имеющие рекуррентные связи между скрытыми блоками.
2. Сети, порождающие выход на каждом временном шаге, и имеющие рекуррентные связи между выходами на одном шаге и скрытыми блоками на следующем.
3. Сети с рекуррентными связями между скрытыми блоками, которые читают последовательность целиком, а затем порождают единственный выход.

### Сети с рекуррентными связями между скрытыми блоками
Пример для задачи классификации: пусть $W$, $U$, $V$ -- матрицы весовых коэффициентов сети, тогда на каждом шаге:

1. преобразуем входной сигнал и прошлое состояние в новое состояние сети:
$$a^{(t)} = b + Wh^{(t-1)} + U x^{(t)}$$
$$h^{(t)} = th(a^{(t)})$$

2. вычисляем выход сети
$$o^{(t)} = c + V h^{(t)}$$
$$\hat y^{(t)} = softmax(o^{(t)})$$

### Граф вычисления функции потерь сети и его развертка

Для обучения требуется сначала развернуть всю последовательность, затем выполнить обратное распространение ошибки.

![Граф вычислений функции потерь](img/rnn/rnn1.png)
(Гудфеллоу Я. и др. "Глубокое обучение").

### Сети с рекурсией между выходами и скрытыми слоями

![Граф вычислений функции потерь](img/rnn/rnn2.png)
(Гудфеллоу Я. и др. "Глубокое обучение").

Сеть менее мощная (способна обучиться меньшему числу функций):
 * выход сети $o$ на предыдущем шаге - единственная информация, которую можно передавать в будущее;
 * выход сети $o$ - целевая переменная, т.е. сеть обучается возвращать $o$, сложно включить полезные сведения о состоянии сети для передачи их в будущее (в $o$ отсутствует важная информация о прошлом).
 
Не требует развертки во времени всей последовательности:
 * можно обучать каждый шаг по отдельности;
 * параллельность обработки.

### Сети с единственным выходом в конце последовательности


![Граф вычислений функции потерь](img/rnn/rnn3.png)
(Гудфеллоу Я. и др. "Глубокое обучение").

## Пример: создание рекуррентной сети в TensorFlow

 * Создадим рекуррентную сеть, способную обрабатывать последовательности.
 * Будем работать с низкоуровневыми инструментами TensorFlow.

Сеть реализует в нейронах, отвечающих за состояние, следующую функцию:

$$
h^{(t)} = th(W_x x^{(t)} + W_h h^{(t-1)} + b)
$$
$W_x$, $W_h$ и $b$ - обучаемые веса и пороги, общие для всех временных шагов.

В основе кода лежит [данная реализация.](https://github.com/Hezi-Resheff/Oreilly-Learning-TensorFlow/blob/master/05__text_and_visualizations/vanilla_rnn_with_tfboard.py)


#### Задача: классификация изображений MNIST

Изображение можно рассматривать, как последовательность:

 * каждое изображение - последовательность строк (или столбцов);
 * 28х28 пиксельное изображение - это последовательность из 28-ми элементов, каждый элемент - вектор из 28-ми пикселей;
 * пространственную структуру изображения можно заменить на пространственно-временную.
 
Создадим сеть, которая на вход принимает столбцы пикселей один за другим: 0, 1, ..., 27. После получения последнего столбца сеть должна выдать номер класса (цифру на изображении).

Имортируем необходимые библиотеки, считываем данные.

In [ ]:
import tensorflow as tf 
import numpy as np 
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt 
%matplotlib inline  

import sys

DATA_DIR = '/tmp/data' if not 'win32' in sys.platform else "c:\\tmp\\data"
STEPS = 1000
MINIBATCH_SIZE = 50
mnist = input_data.read_data_sets(DATA_DIR, one_hot=True)

Одно из изображений.

In [ ]:
IMAGE_IX_IN_DATASET = 0

first_img = mnist.train.images[IMAGE_IX_IN_DATASET].reshape(28, 28)
first_lbl = mnist.train.labels[IMAGE_IX_IN_DATASET].argmax()

plt.figure()
plt.imshow(first_img, cmap='gray')
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title("Вообще-то это должно быть {}".format(first_lbl))

Определим параметры, с которыми будет работать сеть:
 * размер элемента (число пикселей в колонке);
 * длина последовательности;
 * число классов;
 * размер минипакета для обучения;
 * число нейронов, отвечающих за вектор состояния. 

In [ ]:
element_size = 28
time_steps = 28
num_classes = 10
batch_size = 128
hidden_layer_size = 128

Параметры для хранения входов и выходов сети.

In [ ]:
_inputs = tf.placeholder(tf.float32,
                         shape=[None, time_steps, element_size],
                         name='inputs')
y = tf.placeholder(tf.float32, shape=[None, num_classes], name='labels')

Веса входного слоя и слоя состояний:

In [ ]:
with tf.name_scope('rnn_weights'):
    with tf.name_scope("W_x"):
        Wx = tf.Variable(tf.zeros([element_size, hidden_layer_size]))
    with tf.name_scope("W_h"):
        Wh = tf.Variable(tf.zeros([hidden_layer_size, hidden_layer_size]))
    with tf.name_scope("Bias"):
        b_rnn = tf.Variable(tf.zeros([hidden_layer_size]))

Вычисление нового состояния по входному сигналу и вектору старых состояний:

In [ ]:
def rnn_step(previous_hidden_state, x):
    current_hidden_state = tf.tanh(
            tf.matmul(previous_hidden_state, Wh) +
            tf.matmul(x, Wx) + b_rnn)
    
    return current_hidden_state

Развертка слоя состояний во времени:

In [ ]:
# Осуществим развертку функцией scan
# Текущие размерности данных: (batch_size, time_steps, element_size)
processed_input = tf.transpose(_inputs, perm=[1, 0, 2])
# Новые размерности данных: (time_steps, batch_size, element_size)


initial_hidden = tf.zeros([batch_size, hidden_layer_size])
# Развертка во времени
all_hidden_states = tf.scan(rnn_step,
                            processed_input,
                            initializer=initial_hidden, name='states')

Определим веса и пороги выходного слоя:

In [ ]:
with tf.name_scope('linear_layer_weights') as scope:
    with tf.name_scope("W_linear"):
        Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes],
                                             mean=0, stddev=.01))
    with tf.name_scope("Bias_linear"):
        bl = tf.Variable(tf.truncated_normal([num_classes],
                                             mean=0, stddev=.01))

Определим функцию выхода сети:

In [ ]:
def get_linear_layer(hidden_state):
    return tf.matmul(hidden_state, Wl) + bl

In [ ]:
with tf.name_scope('linear_layer_weights') as scope:
    # Рассчитаем выходы сети для каждого момента времени
    all_outputs = tf.map_fn(get_linear_layer, all_hidden_states)
    # Нас интересует в основном последний ответ-- h_28
    output = all_outputs[-1]

# Функция потерь
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=y))

# Отпимизация
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1.5e-3).minimize(cross_entropy)
    # train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

# Точность в процентах
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(output, 1))
    accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100


Тестовые данные:

In [ ]:
test_data = mnist.test.images[:batch_size].reshape((-1, time_steps, element_size))
test_label = mnist.test.labels[:batch_size]

Вычислительный граф определен. Запустим расчеты на этом графе.

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(5000):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Переформатируем данные, чтобы получтить 28 элементов по 28 пикселей
            batch_x = batch_x.reshape((batch_size, time_steps, element_size))
            sess.run([train_step], feed_dict={_inputs: batch_x, y: batch_y})

            if i % 100 == 0:
                acc, loss, = sess.run([accuracy, cross_entropy],
                                      feed_dict={_inputs: batch_x,
                                                 y: batch_y})
                print("Iter " + str(i) + ", Потеря на пакете= " +
                      "{:.6f}".format(loss) + ", Точность (обуч.)= " +
                      "{:.5f}".format(acc))
            if i % 100 == 0:
                # Вычисляем точность для 128 примеров
                acc = sess.run([accuracy],
                                        feed_dict={_inputs: test_data,
                                                   y: test_label})

    test_acc = sess.run(accuracy, feed_dict={_inputs: test_data,
                                             y: test_label})
print("Точность (тест):", test_acc)